In [1]:
import numpy as np
import librosa
from util.ioUtil import *
import librosa.display
from matplotlib import pyplot as plt
import soundfile as sf
import cv2 as cv

# Test of using beat extraction from librosa

In [2]:
test_file_path = "F:/MASC/video_timing_interaction/songs/"
file_name = "whithers_face.mp4"
drum_sample_path = "F:/MASC/video_timing_interaction/drum_samples/"
drum_file_name = "highHatclosed0.mp3"

In [3]:
# get_audio_from_video(file_name, test_file_path)

In [40]:
file = librosa.load(test_file_path+file_name[:-3]+"mp3", sr=44100)
sr = file[1]
y = file[0]

In [41]:
high_hat = librosa.load(drum_sample_path+drum_file_name, sr=44100)[0]
drum_track = np.zeros(y.shape)


In [42]:
tempo, beats = librosa.beat.beat_track(y=y, sr=sr, tightness=1000)

In [43]:
t = librosa.frames_to_time(beats, sr=sr)
for i in range(0, len(t)):
    onset_time = int(t[i] * 44100)
    drum_track[onset_time:onset_time+high_hat.shape[0]] = high_hat
sf.write(test_file_path + 'drum_rhythm.wav', drum_track, 44100, subtype='PCM_24')

# Just using onset detection on singing

In [3]:
test_file_path = "F:/MASC/video_timing_interaction/songs/"
file_name = "piah.wav"
video_file_name = "piah.mp4"
drum_sample_path = "F:/MASC/video_timing_interaction/drum_samples/"
drum_file_name = "highHatclosed0.mp3"


In [4]:
y, sr = librosa.load(test_file_path+file_name, sr=16000)
beat_only = librosa.feature.melspectrogram(y, sr=sr)
beat_only = librosa.stft(y, hop_length=256, n_fft=1024)

high_hat = librosa.load(drum_sample_path+drum_file_name, sr=16000)[0]
drum_track = np.zeros(y.shape)

ts = (512.0/sr) * np.arange(1, beat_only.shape[1]+1) + 512.0/sr/2

In [5]:
onset_default = librosa.onset.onset_detect(y=y, sr=sr, hop_length=256,
                                           units='frames')

In [6]:
t = onset_default
for i in range(0, len(t)-1):
    onset_time = int(t[i] * 256)
    drum_track[onset_time:onset_time+high_hat.shape[0]] = high_hat
sf.write(test_file_path + 'onset_only_drum_rhythm.wav', drum_track, sr, subtype='PCM_24')

In [7]:
cap = cv.VideoCapture(cv.samples.findFile(test_file_path+video_file_name))
ret, frame1 = cap.read()
prvs_img = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
counter = 0
flow_out = [] 
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next_img = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs_img, next_img, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow_out.append(flow)
    prvs_img = next_img
    counter = counter + 1


No frames grabbed!


In [38]:
t = 0
flow_t = flow_out[t]
vert_up_dir = np.ones(flow_t.shape)
vert_up_dir[:, :, 1] = vert_up_dir[:, :, 1] * 0
# compute angle of the vector by first computing
# dot product between flow direction and vertical
# up, then dividing by magnitude of flow_t
dot_product = (vert_up_dir * flow_t).sum(axis=2) 
magnitude = np.sqrt((flow_t * flow_t).sum(axis=2))
magnitude = np.where(magnitude==0, 1, magnitude)
dire = dot_product / magnitude + np.pi/2

In [40]:
print(dire.min())
bin_a = np.floor(a/np.pi * N_bin)

0.39419479077826436


In [41]:
0.39*180/np.pi

22.345354010102106